<a href="https://colab.research.google.com/github/benbaz-2/comp551/blob/main/assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3

## Installations

In [10]:
!pip install medmnist

In [11]:
import numpy as np

## Load dataset

In [12]:
from medmnist import OrganAMNIST
train_dataset = OrganAMNIST(split='train', download=True)
test_dataset = OrganAMNIST(split='test', download=True)

Using downloaded and verified file: /root/.medmnist/organamnist.npz
Using downloaded and verified file: /root/.medmnist/organamnist.npz


In [13]:
# Put the data in numpy arrays
x_train, y_train = train_dataset.imgs, train_dataset.labels
x_test, y_test = test_dataset.imgs, test_dataset.labels

In [18]:
# vectorize x
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)
train_dataset


Dataset OrganAMNIST of size 28 (organamnist)
    Number of datapoints: 34561
    Root location: /root/.medmnist
    Split: train
    Task: multi-class
    Number of channels: 1
    Meaning of labels: {'0': 'bladder', '1': 'femur-left', '2': 'femur-right', '3': 'heart', '4': 'kidney-left', '5': 'kidney-right', '6': 'liver', '7': 'lung-left', '8': 'lung-right', '9': 'pancreas', '10': 'spleen'}
    Number of samples: {'train': 34561, 'val': 6491, 'test': 17778}
    Description: The OrganAMNIST is based on 3D computed tomography (CT) images from Liver Tumor Segmentation Benchmark (LiTS). It is renamed from OrganMNIST_Axial (in MedMNIST v1) for simplicity. We use bounding-box annotations of 11 body organs from another study to obtain the organ labels. Hounsfield-Unit (HU) of the 3D images are transformed into gray-scale with an abdominal window. We crop 2D images from the center slices of the 3D bounding boxes in axial views (planes). The images are resized into 1×28×28 to perform multi-cla

In [15]:
# center and normalize data
mean_train = np.mean(x_train, axis=1, keepdims=True)
std_train = np.std(x_train, axis=1, keepdims=True)
x_train = (x_train - mean_train) / std_train

mean_test = np.mean(x_test, axis=1, keepdims=True)
std_test = np.std(x_test, axis=1, keepdims=True)
x_test = (x_test - mean_test) / std_test

## Implement a MLP

In [20]:
# notes
# I will use a predefined input size and output size

class MLP:
  def __init__(self, hidden_activation='ReLU', output_activation='Softmax', hidden_sizes=[256, 256]):
    self.hidden_activation = hidden_activation
    self.output_activation = output_activation
    self.hidden_sizes = hidden_sizes
    self.input_size = 784
    self.output_size = 11
    layer_sizes = [self.input_size] + self.hidden_sizes + [self.output_size]
    self.weights = [np.random.uniform(- (6 / self.input_size) ** 0.5, (6 / self.input_size) ** 0.5,
                    (layer_sizes[i], layer_sizes[i+1])) for i in range(len(layer_sizes) - 1)]
    self.biases = [np.zeros((1, layer_sizes[i+1])) for i in range(len(layer_sizes) - 1)]

In [27]:
model = MLP()
model.weights[2].shape

(256, 11)